## 1. Subreddit Selection

In [1]:
import praw
import pandas as pd

### Initialize Reddit API


In [2]:
client_id = 'XqvMFAQqrS1xWiUpCcbSOg'
client_secret = '1WDnClQ3eo8Luzug2FSmZ6x9Aj1Ytg'
user_agent = 'ios:MyRedditApp:v1.0 (by /u/sk_shuanq)'

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Test if connected
print(reddit.read_only)  # Expected True

try:
    # Test with subreddit: python
    subreddit = reddit.subreddit('python')
    print(f"Successfully connected! Subreddit title: {subreddit.title}")
    print(f"Read-only mode: {reddit.read_only}")  # True Expected
except Exception as e:
    print(f"Connection failed: {e}")

True
Successfully connected! Subreddit title: Python
Read-only mode: True


In [3]:
keywords = [
    'libertarian', 'conservative', 'politics', 'republican', 'democrat', 
    'news', 'view', 'gun control', 'lgbt', 'abortion', 'vaccine', 
    'climate change', 'immigration', 'tax', 'racism'
]

### Search for Keyword

In [4]:
# List to store results
subreddit_list = []

# Search for subreddits containing each keyword
for keyword in keywords:
    print(f"Searching for keyword: {keyword}")
    try:
        # Fetch results with the keyword
        search_results = reddit.subreddits.search_by_name(keyword, include_nsfw=False)  # filter Not Safe For Work content
        
        # Collect subreddit details
        temp_results = []
        for subreddit in search_results:
            temp_results.append({
                "keyword": keyword,
                "subreddit": subreddit.display_name,
                "subscribers": subreddit.subscribers,
                "description": subreddit.public_description
            })
        
        # Sort by subscribers and take the top 4
        temp_results = sorted(temp_results, key=lambda x: x["subscribers"], reverse=True)[:4]
        
        # Append to main list
        subreddit_list.extend(temp_results)
    except Exception as e:
        print(f"Error searching for keyword '{keyword}': {e}")


Searching for keyword: libertarian
Searching for keyword: conservative
Searching for keyword: politics
Searching for keyword: republican
Searching for keyword: democrat
Searching for keyword: news
Searching for keyword: view
Searching for keyword: gun control
Searching for keyword: lgbt
Searching for keyword: abortion
Searching for keyword: vaccine
Searching for keyword: climate change
Searching for keyword: immigration
Searching for keyword: tax
Searching for keyword: racism


### Convert and Save

In [5]:
# Convert results to DataFrame
df = pd.DataFrame(subreddit_list)

# Display the DataFrame
print("\nTop Subreddits by Keyword:")
print(df)

# Save to CSV for future reference
file_path_subreddit_raw = "data/raw-data/subreddits_raw.csv"
df.to_csv(file_path_subreddit_raw, index=False)


Top Subreddits by Keyword:
           keyword              subreddit  subscribers  \
0      libertarian            Libertarian       503241   
1      libertarian        libertarianmeme       147424   
2      libertarian        LibertarianLeft        14930   
3      libertarian   LibertarianSocialism        13957   
4     conservative           Conservative      1154369   
5     conservative          conservatives       108335   
6     conservative  conservativeterrorism        85102   
7     conservative      ConservativeMemes        75440   
8         politics               politics      8687246   
9         politics             ukpolitics       507858   
10        politics     AustralianPolitics       236946   
11        politics         CanadaPolitics       231836   
12      republican             Republican       206488   
13      republican            republicans        19836   
14      republican       RepublicanValues        19490   
15      republican   RepublicanPedophiles   

### Cleaning Subreddit

In [6]:

df_raw = pd.read_csv(file_path_subreddit_raw)

# Remove irrelevant subriddts
subreddits_to_remove = ['libertarianmeme','ConservativeMemes','ukpolitics','AustralianPolitics','CanadaPolitics','NewsOfTheStupid','NewSkaters','TradingView','remoteviewing','CrossView','lgbtmemes','climate_science','ImmigrationCanada','taxPH','IndiaTax','taxpros','AccidentalRacism']
df_filtered = df_raw[~df_raw['subreddit'].isin(subreddits_to_remove)]

# Add valuable subriddts the search missed
new_rows = [
    {"keyword": "conservative", "subreddit": "AskTrumpSupporters", "subscribers": 92000, "description": "Q&A subreddit to understand Trump supporters, their views, and the reasons behind those views. Debates are discouraged."},
    {"keyword": "conservative", "subreddit": "AskConservatives", "subscribers": 23000, "description": "Welcome to r/AskConservatives! A sub to ask conservatives questions with the intent of better understanding Conservatism and conservative perspectives. The sub tends to have a focus on US politics, but we welcome all Canadian, UK, Aus, and European topics and users, as well as world politics in general. Open discussions are strongly encouraged. Please remember to keep things civil and respect others even when you disagree."},
    {"keyword": "libertarian", "subreddit": "AskLibertarians", "subscribers": 12000, "description": "A friendly place to learn about, critique, and question libertarians and their views. r/AskLibertarians is for any questions about the philosophy of libertarianism, libertarian movements and traditions, libertarian opinions on certain situations or current events, or anything else you feel is relevant. No question is too basic (or advanced!) to ask, so don't be shy. Subscribe :)"},
    {"keyword": "politics", "subreddit": "moderatepolitics", "subscribers": 298000, "description": "Restore Sanity in Politics!This is NOT a politically moderate subreddit! It IS a political subreddit for moderately expressed opinions and civil discourse. If you are looking for civility, moderation and tolerance come on in!"},
]

df_new = pd.DataFrame(new_rows)
df_updated = pd.concat([df_filtered, df_new], ignore_index=True)

# Remove subriddts that have less than 10000 subscribers
df_subreddit = df_updated[df_updated['subscribers'] >= 10000]
print(f"Total Subreddits: {len(df_subreddit)}")

# Save the final selected subriddts
file_path_subreddit = "data/processed-data/subreddits.csv"
df_subreddit.to_csv(file_path_subreddit, index=False)
df_subreddit.head(6)


Total Subreddits: 35


,keyword,subreddit,subscribers,description
0,libertarian,Libertarian,503241,"Welcome to /r/Libertarian, a subreddit to disc..."
1,libertarian,LibertarianLeft,14930,"Liberty is the mother, not the daughter, of or..."
2,libertarian,LibertarianSocialism,13957,A community for the discussion of news and lin...
3,conservative,Conservative,1154369,The largest conservative subreddit.\nhttps://d...
4,conservative,conservatives,108335,"Conservatism (from conservare, ""to preserve"") ..."
5,conservative,conservativeterrorism,85102,Documenting conservative terrorist actions and...
